In [2]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
from openpyxl import load_workbook



In [3]:
# cargar  archivos

df_items = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='ITEMS')
df_consumos = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='CONSUMOS',index_col='Año')
df_lista_materiales = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='lista_materiales')
df_inventarios = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='INVENTARIOS')
df_pedidos = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='Pedidos_clientes')
df_solicitudes = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='solicitudes')
df_OCC = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='ordenes_compra')
df_OCC_HISTORICO = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION\consumos/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='HISTORICO OCC')


TABLA FRECUENCIA DE CONSUMOS MES A MES

In [4]:
# INFORMACION DE LOS ITEMS 
filtro_año = 2024

df_consumos['Fecha'] = pd.to_datetime(df_consumos['Fecha'], errors='coerce')
df_consumos['Mes-Año'] = df_consumos['Fecha'].dt.strftime('%b-%y')


df_consumos_filtro=df_consumos.copy()
df_consumos_filtro=df_consumos_filtro.loc[filtro_año]

df_consumos_filtro= df_consumos_filtro.groupby(['Mes-Año','Referencia','Desc. item','Bodega'])['Salidas (inv.)'].sum().reset_index()     
df_consumos_mes_año = df_consumos_filtro.pivot_table(index=['Referencia','Desc. item','Bodega'],columns='Mes-Año',values='Salidas (inv.)',aggfunc='sum').reset_index()
df_consumos_mes_año = df_consumos_mes_año.fillna(0)
df_consumos_mes_año 

columnas_consmo = df_consumos_mes_año.columns[3:]

df_consumos_mes_año['Frecuencia_consumo'] = (df_consumos_mes_año[columnas_consmo] > 0).sum(axis=1)

df_consumos_mes_año  #ESTA TABLA MANEJA LA FRECUENCIA DE CONSUMOS DE CADA ITEM POR BODEGA







Mes-Año,Referencia,Desc. item,Bodega,Apr-24,Aug-24,Dec-24,Feb-24,Jan-24,Jul-24,Jun-24,Mar-24,May-24,Nov-24,Oct-24,Sep-24,Frecuencia_consumo
0,3,CARTON SINGLE FACE KILOS,MP001,354.0,0.0,0.0,167.08,0.0,0.0,0.0,283.73,176.0,0.0,0.0,0.0,4
1,3,CARTON SINGLE FACE KILOS,MP002,839.0,0.0,0.0,711.19,337.0,0.0,0.0,1017.00,17.0,0.0,0.0,0.0,5
2,8,CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7,MP001,46.0,68.0,57.0,55.00,98.0,68.0,56.0,41.00,30.0,79.0,61.0,86.0,12
3,8,CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7,MP002,167.0,193.0,196.0,233.00,125.0,123.0,125.0,104.00,118.0,234.0,424.0,45.0,12
4,9,CINTA EMPAQUE 2 * 100 M TOLDOS TOSCANI,MP002,30.0,23.0,33.0,27.00,18.0,15.0,10.0,22.00,28.0,14.0,17.0,6.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,VIGAAC03,VIGA EN ACERO 150X100,MP002,0.0,0.0,0.0,0.00,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,1
3994,VIGAAC04,VIGA EN ACERO 100X50,MP002,0.0,0.0,0.0,1.00,0.0,0.0,2.0,0.00,10.0,0.0,0.0,0.0,3
3995,VIGAAC05 ESP01,VIGA EN ACERO 250x100 CAL 5mm 8M,MP002,1.0,0.0,0.0,0.00,1.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,3
3996,VIGAAC05 ESP02,VIGA EN ACERO 250x100 CAL 5mm 5.7M,MP002,0.0,0.0,0.0,0.00,0.0,0.0,0.0,3.00,0.0,0.0,0.0,0.0,1


TABLA CONSUMOS MAXIMOS-MINIMOS


In [5]:
df_total_consumos = df_consumos.copy()
df_total_consumos_salida_pron = df_consumos.copy()
def winsorize_group(serie):
    return winsorize(serie, limits=[0.05, 0.05])  # Ajusta el umbral si es necesario

# Aplicar por cada ITEM
df_total_consumos_salida_pron['Salida_pron'] = (
    df_total_consumos.groupby(['Referencia','Bodega'])['Salidas (inv.)']
    .transform(lambda x: winsorize_group(x))
)


df_total_consumos = df_total_consumos.groupby(['Año','Referencia','Desc. item','Bodega']).agg(

    Total_consumo=('Salidas (inv.)','sum'),
    Consumo_maximo=('Salidas (inv.)','max'),
    Consumo_minimo=('Salidas (inv.)','min'),
    Consumo_promedio=('Salidas (inv.)','mean'),
    desviacion_estandar=('Salidas (inv.)','std')

).reset_index()

df_total_consumos.set_index('Año', inplace=True)



df_total_consumos=df_total_consumos.loc[filtro_año]

df_total_consumos  #ESTA TABLA MANEJA EL TOTAL DE CONSUMOS DE CADA ITEM POR BODEGA

df_total_consumos = df_total_consumos.merge(df_total_consumos_salida_pron[['Referencia', 'Desc. item', 'Bodega','Salida_pron']], how='left',on=['Referencia','Desc. item','Bodega'])

print(df_total_consumos.columns)


df_total_consumos=df_total_consumos.groupby(['Referencia','Desc. item','Bodega']).agg({
    'Total_consumo':'mean',
    'Salida_pron':['min','max','mean','std','median']
    
    
    }
)

df_total_consumos.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_total_consumos.columns]

print(df_total_consumos)
df_total_consumos
df_total_consumos.to_excel('val.xlsx')



Index(['Referencia', 'Desc. item', 'Bodega', 'Total_consumo', 'Consumo_maximo',
       'Consumo_minimo', 'Consumo_promedio', 'desviacion_estandar',
       'Salida_pron'],
      dtype='object')
                                                               Total_consumo_mean  \
Referencia     Desc. item                              Bodega                       
3              CARTON SINGLE FACE KILOS                MP001               980.81   
                                                       MP002              2921.19   
8              CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7 MP001               745.00   
                                                       MP002              2087.00   
9              CINTA EMPAQUE  2 * 100 M TOLDOS TOSCANI MP002               243.00   
...                                                                           ...   
VIGAAC03       VIGA EN ACERO 150X100                   MP002                 1.00   
VIGAAC04       VIGA EN ACERO 100X50       

In [90]:
df_items_consolidado = df_items.copy()

df_items_consolidado= df_items_consolidado.merge(df_total_consumos,how='left', on = ['Referencia','Desc. item','Bodega'])
print(df_items_consolidado.columns)

df_items_consolidado= df_items_consolidado.merge(df_inventarios[['Referencia','Desc. item','Bodega','Cant. disponible']],how='left', on=['Referencia','Desc. item','Bodega'])




df_items_consolidado =df_items_consolidado.merge(df_OCC[['Referencia','Desc. item','Bodega','Cant. pendiente']],how='left',on=['Referencia','Desc. item','Bodega'])



df_items_consolidado = df_items_consolidado.fillna(0)
df_items_consolidado =df_items_consolidado.merge(df_solicitudes[['Referencia','Desc. item','Bodega','Cant. pendiente']],how='left',on=['Referencia','Desc. item','Bodega'])



df_items_consolidado = df_items_consolidado.merge(df_lista_materiales[['Referencia','Desc. item','Bodega','Pedidos Acom','Inv.Proyectado']],how='left',on=['Referencia','Desc. item','Bodega'])

print(df_items_consolidado)

df_items_consolidado.to_excel('valY.xlsx')

df_items_consolidado = df_items_consolidado.groupby(
    ['Referencia', 'Desc. item', 'CATEGORIA', 'TIPO', 'ORIGEN', 'LINEA', 'Bodega', 'U.M.','STATUS'],
    as_index=False  # Evita que las columnas de agrupación se vuelvan índice
).agg({
    'Total_consumo_mean':'mean',
    'Salida_pron_min':'mean',
    'Salida_pron_max':'mean',
    'Salida_pron_mean':'mean',
    'Salida_pron_std':'mean',
    'Salida_pron_median':'mean',
    'Cant. disponible':'mean',
    'Cant. pendiente_x':'mean', 
    'Cant. pendiente_y':'mean',
    'Pedidos Acom':'mean',
    'Inv.Proyectado':'mean',
    'Lead time esperado':'mean'
    }
).rename(columns={'Total_consumo_mean':'Consumos_2024','Salida_pron_min':'Consumo_min','Salida_pron_max':'Consumo_max','Salida_pron_mean':'Consumo_prom','Salida_pron_std':'Desv_consumo','Salida_pron_median':'Consumo_mediana','Cant. pendiente_x':'Cant. pendiente_OCC','Cant. pendiente_y':'Cant. pendiente_solicitudes','Cant. Total': 'Requerimientos_pedidos'})


#df_items_consolidado.to_excel('consumoI.xlsx', index=False)

Index(['Referencia', 'Desc. item', 'CATEGORIA', 'TIPO', 'ORIGEN', 'LINEA',
       'Bodega', 'U.M.', 'STATUS', 'Lead time esperado', 'Total_consumo_mean',
       'Salida_pron_min', 'Salida_pron_max', 'Salida_pron_mean',
       'Salida_pron_std', 'Salida_pron_median'],
      dtype='object')
      Referencia                                Desc. item  \
0        PZAC159             SOPORTE MOTOR PERGOLAM GALAXY   
1        PZAC159             SOPORTE MOTOR PERGOLAM GALAXY   
2        PZCH125    PERFIL EN SILICONA SELLE LAMA PERGOLAM   
3          26735        100W OUTPUT SWITCHING FUENTE POWER   
4          22537  10393548 GRIS ANTRACITA GRAN MATE X25 KG   
...          ...                                       ...   
18424       5345                 ZAPATO TENNIS AZUL CORDON   
18425       5345                 ZAPATO TENNIS AZUL CORDON   
18426       3076                                ZUNCHADORA   
18427      12831                         ZUNCHO * 500 MTRS   
18428      12831            

In [92]:

base_occ_filtrado = df_OCC_HISTORICO[
    df_OCC_HISTORICO['Fecha aprobación solicitud'].notna() &
    df_OCC_HISTORICO['Fecha cumplido'].notna()
].copy()



#base_occ_filtrado['Fecha orden'] = pd.to_datetime(base_occ_filtrado['Fecha orden'], errors='coerce')



base_occ_filtrado["Fecha orden"] = pd.to_datetime(base_occ_filtrado["Fecha orden"], errors="coerce")


base_occ_2024 = base_occ_filtrado[base_occ_filtrado["Fecha orden"].dt.year == 2024]

base_occ_2024["Frecuencia_mes_occ"] = base_occ_filtrado["Fecha orden"].dt.to_period("M")

frecuencia_mes_occ = base_occ_2024.groupby(["Referencia","Bodega"])["Frecuencia_mes_occ"].nunique().reset_index()

frecuencia_ordenes = base_occ_2024[['Fecha orden','Referencia','Bodega','Cant. ordenada','Frecuencia_mes_occ']].copy()



frecuencia_ordenes["Frecuencia_mes_occ"] = frecuencia_ordenes["Frecuencia_mes_occ"].astype(int)




print(type(frecuencia_mes_occ))

frecuencia_mes_occ.to_excel('frecuencia_mes_occ.xlsx')




base_occ_filtrado['Lead time'] = base_occ_filtrado['Fecha cumplido'] - base_occ_filtrado['Fecha aprobación solicitud']
base_occ_filtrado['lead time (dias)'] = base_occ_filtrado['Lead time'].dt.days
base_occ_filtrado=base_occ_filtrado[['Referencia','Desc. item','Bodega','lead time (dias)','Cant. ordenada']]
base_occ_filtrado = base_occ_filtrado[(base_occ_filtrado['Bodega'].isin(['MP001','MP002']))]


base_occ_filtrado.to_excel('baseopm.xlsx')



base_occ_filtrado['Ordenes'] = (
    base_occ_filtrado.groupby(['Referencia','Bodega'])['Cant. ordenada']
    .transform(lambda x: winsorize_group(x))
)


base_occ_filtrado = base_occ_filtrado.merge(df_items_consolidado[['Referencia', 'Desc. item', 'Bodega','Lead time esperado']], how='left',on=['Referencia','Desc. item','Bodega'])

print(frecuencia_mes_occ.head())


base_occ_filtrado = base_occ_filtrado.merge(frecuencia_mes_occ[['Referencia','Bodega','Frecuencia_mes_occ']], how='left',on=['Referencia','Bodega'])

print(base_occ_filtrado.head())



base_occ_filtrado['lead time winsorized'] = winsorize(base_occ_filtrado['lead time (dias)'], limits=[0.05,0.05])
# 2. Calcular media y desviación estándar de los datos ajustados
mu = base_occ_filtrado['lead time winsorized'].mean()
sigma= base_occ_filtrado['lead time winsorized'].std()

# 3. Normalización: Centrar en 10 días con desviación ajustada a 5 días
sigma_nuevo = 5
sigma_nuevo_sg = 4

base_occ_filtrado['lead time (dias) normalized'] = base_occ_filtrado['Lead time esperado'] + ((base_occ_filtrado['lead time winsorized'] - mu) / sigma) * sigma_nuevo
base_occ_filtrado['lead time (dias) normalized_sig4'] = base_occ_filtrado['Lead time esperado'] + ((base_occ_filtrado['lead time winsorized'] - mu) / sigma) * sigma_nuevo_sg

base_occ_filtrado = base_occ_filtrado.groupby(
    ['Referencia', 'Desc. item','Bodega'],
    as_index=False  # Evita que las columnas de agrupación se vuelvan índice
).agg({
    'Ordenes':['mean','min','max'],
    'Frecuencia_mes_occ':'mean',
    'lead time (dias)':'mean',
    'Lead time esperado':'mean',
    'lead time winsorized':'mean',
    'lead time (dias) normalized':'mean', 
    'lead time (dias) normalized_sig4':['mean','max','median','std'],
    
    
    }
)




base_occ_filtrado.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in base_occ_filtrado.columns]



base_occ_filtrado.to_excel('base_occ_filtrado.xlsx')

C:\Users\JORGE CONTRERAS\AppData\Local\Temp\ipykernel_12012\4082143038.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_occ_2024["Frecuencia_mes_occ"] = base_occ_filtrado["Fecha orden"].dt.to_period("M")


<class 'pandas.core.frame.DataFrame'>
  Referencia Bodega  Frecuencia_mes_occ
0          3  MP001                   3
1          3  MP002                   3
2          8  MP001                   5
3          8  MP002                   7
4          9  MP002                   3
  Referencia                                Desc. item Bodega  \
0    PZCH072                       ABRAZADERA DE NYLON  MP002   
1     101827               POLEA (627Z) 32MM PERGOFLEX  MP002   
2     101846                 TUERCA BUJE 10mm PERGOTEK  MP002   
3       3151  CORREA DENTADA 19mm PERGOLAS RETRACTILES  MP002   
4    PZCH070     EJE LAMA MAXI PLAM ANTISI HUECO Ø12mm  MP002   

   lead time (dias)  Cant. ordenada  Ordenes  Lead time esperado  \
0                 9           100.0    100.0               120.0   
1               169           200.0    200.0               120.0   
2               169           100.0    100.0               120.0   
3               169          2500.0   2500.0               

In [94]:
dias_operacion=300
df_base_final = df_items_consolidado.merge(base_occ_filtrado, how='left', on=['Referencia','Desc. item','Bodega'])
#df_base_final =df_base_final.drop(columns=['Lead time esperado_y'])
df_base_final =df_base_final.merge(df_consumos_mes_año[['Referencia','Desc. item','Bodega','Frecuencia_consumo']],how='left',on=['Referencia','Desc. item','Bodega'])

print(df_base_final.columns)
df_base_final=df_base_final.drop(columns=['lead time (dias)mean','Lead time esperadomean','lead time winsorizedmean','lead time (dias) normalizedmean'])
df_base_final=df_base_final.fillna(0)
df_base_final['Consumo_diario']=df_base_final['Consumos_2024']/dias_operacion
df_base_final['Consumo_diario_mes']=df_base_final['Consumos_2024']/df_base_final['Frecuencia_consumo']
df_base_final['Consumo_diario_diario']=df_base_final['Consumo_diario_mes']/25
df_base_final['Stock_Seguridad'] = (((df_base_final['Consumo_max']/25)*df_base_final['lead time (dias) normalized_sig4median'])-((df_base_final['Consumo_prom']/25)*df_base_final['lead time (dias) normalized_sig4mean']))
df_base_final['Punto_reorden'] = (df_base_final['lead time (dias) normalized_sig4mean']*df_base_final['Consumo_diario'])+df_base_final['Stock_Seguridad']
df_base_final['nivel_minimo_inv'] = df_base_final['Punto_reorden'] - df_base_final['Stock_Seguridad']
df_base_final['nivel_maximo_inv'] = df_base_final['Punto_reorden'] + (df_base_final['Ordenesmean']-(df_base_final['Consumo_diario']*df_base_final['lead time (dias) normalized_sig4mean']))
df_base_final['nivel_optimo_solicitar']=(df_base_final['nivel_maximo_inv']+df_base_final['nivel_minimo_inv'])/2
df_base_final['Coonsumo_mensual']=df_base_final['Consumo_diario']*25
df_base_final['Cubrimiento_inv'] = (df_base_final['Cant. disponible']-df_base_final['Pedidos Acom'])/df_base_final['Consumo_diario']
df_base_final=df_base_final.fillna(0)



libro = load_workbook(r'C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/consumos/INDICADORES PLANEACION\BASES_INDICADORES/consumoI.xlsx')

if 'consolidado' in libro.sheetnames:
    del libro['consolidado']

libro.save(r'C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/consumos/INDICADORES PLANEACION\BASES_INDICADORES/consumoI.xlsx')

with pd.ExcelWriter(r'C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/consumos/INDICADORES PLANEACION\BASES_INDICADORES/consumoI.xlsx', engine='openpyxl', mode='a') as writer:
    df_base_final.to_excel(writer, sheet_name='consolidado', index=False)


Index(['Referencia', 'Desc. item', 'CATEGORIA', 'TIPO', 'ORIGEN', 'LINEA',
       'Bodega', 'U.M.', 'STATUS', 'Consumos_2024', 'Consumo_min',
       'Consumo_max', 'Consumo_prom', 'Desv_consumo', 'Consumo_mediana',
       'Cant. disponible', 'Cant. pendiente_OCC',
       'Cant. pendiente_solicitudes', 'Pedidos Acom', 'Inv.Proyectado',
       'Lead time esperado', 'Ordenesmean', 'Ordenesmin', 'Ordenesmax',
       'Frecuencia_mes_occmean', 'lead time (dias)mean',
       'Lead time esperadomean', 'lead time winsorizedmean',
       'lead time (dias) normalizedmean',
       'lead time (dias) normalized_sig4mean',
       'lead time (dias) normalized_sig4max',
       'lead time (dias) normalized_sig4median',
       'lead time (dias) normalized_sig4std', 'Frecuencia_consumo'],
      dtype='object')


In [8]:
df_consumos=df_consumos[df_consumos['Referencia']==6543]
df_consumos

,Mes,Orden interno,Referencia,Fecha,Documento,Desc. item,Bodega,Lote,Ubicación,Entradas (inv.),Salidas (inv.),Mes-Año
Año,,,,,,,,,,,,
2023,2023-07-31,7,6543,2023-07-06,CPM-00032688,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,1.0,Jul-23
2023,2023-07-31,7,6543,2023-07-12,REM-00046214,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,2.0,Jul-23
2023,2023-07-31,7,6543,2023-07-13,CPM-00032776,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,1.0,Jul-23
2023,2023-08-31,8,6543,2023-08-25,REM-00046685,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,8.0,Aug-23
2023,2023-08-31,8,6543,2023-08-26,REM-00046686,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,2.0,Aug-23
...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024-10-31,10,6543,2024-10-25,CPM-00037997,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,3.0,Oct-24
2024,2024-10-31,10,6543,2024-10-28,CPM-00038002,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,2.0,Oct-24
2024,2024-10-31,10,6543,2024-10-28,CPM-00038002,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0.0,5.0,Oct-24
